## Model Compilation Jupyter Notebook Example

This notebook shows the example of model compilation using edgeai-benchmark.

This script uses TIDL to compile a model and output in a format that edgeai-sdk can understand.

jai_benchmark is a python package provided in edgeai-benchmark that provides several functions to assist model compilation.

In [1]:
import os
import tempfile
import argparse
import cv2
from jai_benchmark import *

In [2]:
# the cwd must be the root of the respository
if os.path.split(os.getcwd())[-1] in ('scripts', 'tutorials'):
    os.chdir('../')
#
print(os.environ['TIDL_TOOLS_PATH'])
print(os.getcwd())

/data/ssd/files/a0393608/work/code/ti/edgeai-algo/edgeai-benchmark/tidl_tools
/data/ssd/files/a0393608/work/code/ti/edgeai-algo/edgeai-benchmark


#### Create a temporary directory. 

This is were the compiled artifacts will be stored.

In [3]:
modelartifacts_tempdir = tempfile.TemporaryDirectory()
print(modelartifacts_tempdir)
modelartifacts_custom = os.path.join(modelartifacts_tempdir.name, 'modelartifacts')

<TemporaryDirectory '/tmp/tmp101src05'>


#### Read settings from settings_import_on_pc.yaml

Modify the settings as necessary in the constructor of settings.ConfigSettings()

In [4]:
settings = config_settings.ConfigSettings('./settings_import_on_pc.yaml', 
                modelartifacts_path=modelartifacts_custom,
                calibration_frames=10, calibration_iterations=10, num_frames=100)

work_dir = os.path.join(settings.modelartifacts_path, f'{settings.tensor_bits}bits')
print(f'work_dir = {work_dir}')

work_dir = /tmp/tmp101src05/modelartifacts/32bits


#### Create Dataset Reader classes

Change the dataset paths according to your dataset location

In [5]:
dataset_calib_cfg = dict(
    path=f'{settings.datasets_path}/imagenet/val',
    split=f'{settings.datasets_path}/imagenet/val.txt',
    num_classes=1000,
    shuffle=True,
    num_frames=min(settings.calibration_frames,50000),
    name='imagenet'
)

# dataset parameters for actual inference
dataset_val_cfg = dict(
    path=f'{settings.datasets_path}/imagenet/val',
    split=f'{settings.datasets_path}/imagenet/val.txt',
    num_classes=1000,
    shuffle=True,
    num_frames=min(settings.num_frames,50000),
    name='imagenet'
)

calib_dataset = datasets.ImageClassification(**dataset_calib_cfg)
val_dataset = datasets.ImageClassification(**dataset_val_cfg)

#### Session runtime_options

The default runtime_options can be overriden by passing a runtime_options dict to this function

In [6]:
# choose one session_name depending on the model type
# tflitert for tflite models, onnxrt for onnx models, tvmdlr for mxnet models.
session_name = constants.SESSION_NAME_TFLITERT
#session_name = constants.SESSION_NAME_ONNXRT
#session_name = constants.SESSION_NAME_TVMDLR

session_type = settings.get_session_type(session_name)
runtime_options = settings.get_runtime_options(session_name, is_qat=False)

print(session_type)
print(runtime_options)

<class 'jai_benchmark.sessions.tflitert_session.TFLiteRTSession'>
{'tensor_bits': 32, 'accuracy_level': 1, 'debug_level': 0, 'priority': 0, 'advanced_options:high_resolution_optimization': 0, 'advanced_options:pre_batchnorm_fold': 1, 'advanced_options:calibration_frames': 10, 'advanced_options:calibration_iterations': 10, 'advanced_options:quantization_scale_type': 0, 'advanced_options:activation_clipping': 1, 'advanced_options:weight_clipping': 1, 'advanced_options:bias_calibration': 1, 'advanced_options:channel_wise_quantization': 0, 'advanced_options:output_feature_16bit_names_list': '', 'advanced_options:params_16bit_names_list': '', 'advanced_options:add_data_convert_ops': 3}


In [11]:
preproc_transforms = preprocess.PreProcessTransforms(settings)
postproc_transforms = postprocess.PostProcessTransforms(settings)

# these session cfgs also has some default input mean and scale. 
# if your model needs a difference mean and scale, update the session cfg dict being used with those values
onnx_session_cfg = sessions.get_onnx_session_cfg(settings, work_dir=work_dir)
tflite_session_cfg = sessions.get_tflite_session_cfg(settings, work_dir=work_dir)

#### Create pipeline_configs

pipeline_configs is nothing but a dict with the various model configs that we want to compile

In [9]:
pipeline_configs = {
    'cl-mnv2': dict(
        task_type='classification',
        calibration_dataset=calib_dataset,
        input_dataset=val_dataset,
        preprocess=preproc_transforms.get_transform_tflite(),
        session=session_type(**tflite_session_cfg,
            runtime_options=runtime_options,
            model_path=f'{settings.models_path}/vision/classification/imagenet1k/tf1-models/mobilenet_v2_1.0_224.tflite'),
        postprocess=postproc_transforms.get_transform_classification(),
        metric=dict(label_offset_pred=-1),
        model_info=dict(metric_reference={'accuracy_top1%':71.9})
    ),
}
print(pipeline_configs)

{'cl-mnv2': {'task_type': 'classification', 'calibration_dataset': <jai_benchmark.datasets.image_cls.ImageClassification object at 0x7ff383268588>, 'input_dataset': <jai_benchmark.datasets.image_cls.ImageClassification object at 0x7ff383268518>, 'preprocess': <jai_benchmark.preprocess.PreProcessTransforms object at 0x7ff383268668>, 'session': <jai_benchmark.sessions.tflitert_session.TFLiteRTSession object at 0x7ff383268ba8>, 'postprocess': <jai_benchmark.postprocess.PostProcessTransforms object at 0x7ff3832687b8>, 'metric': {'label_offset_pred': -1}, 'model_info': {'metric_reference': {'accuracy_top1%': 71.9}}}}


#### Model Compilation

This will take a few minutes. Please be patient...

In [10]:
# run the model compliation/import and inference
tools.run_accuracy(settings, work_dir, pipeline_configs)

configs to run: ['cl-mnv2_tflitert_imagenet1k_tf1-models_mobilenet_v2_1.0_224_tflite']
number of configs: 1


TASKS                                                       |   0%|          || 0/1 [00:00<?, ?it/s]

TASKS                                                       |          |     0% 0/1| [< ]
INFO:20220329-160924: starting process on parallel_device - 0
INFO:20220329-160924: model_path - /data/ssd/files/a0393608/work/code/ti/edgeai-algo/edgeai-modelzoo/models/vision/classification/imagenet1k/tf1-models/mobilenet_v2_1.0_224.tflite
INFO:20220329-160924: model_file - /tmp/tmp101src05/modelartifacts/32bits/cl-mnv2_tflitert_imagenet1k_tf1-models_mobilenet_v2_1.0_224_tflite/model/mobilenet_v2_1.0_224.tflite
Download done for /data/ssd/files/a0393608/work/code/ti/edgeai-algo/edgeai-modelzoo/models/vision/classification/imagenet1k/tf1-models/mobilenet_v2_1.0_224.tflite
/tmp/tmp101src05/modelartifacts/32bits/cl-mnv2_tflitert_imagenet1k_tf1-models_mobilenet_v2_1.0_224_tflite/model/mobilenet_v2_1.0_224.tflite

INFO:20220329-160925: running - cl-mnv2_tflitert_imagenet1k_tf1-models_mobilenet_v2_1.0_224_tflite
INFO:20220329-160925: pipeline_config - {'task_type': 'classification', 'calibration_datas

infer : cl-mnv2_tflitert_imagenet1k_tf1-models_mobilenet_v2_|   0%|          || 0/100 [00:00<?, ?it/s]

infer : cl-mnv2_tflitert_imagenet1k_tf1-models_mobilenet_v2_|          |     0% 0/100| [< ]
INFO:20220329-160954: infer completed  - cl-mnv2_tflitert_imagenet1k_tf1-models_mobilenet_v2_1.0_224_tflite - 26 sec

SUCCESS:20220329-160954: benchmark results - {'infer_path': 'cl-mnv2_tflitert_imagenet1k_tf1-models_mobilenet_v2_1.0_224_tflite', 'accuracy_top1%': 79.0, 'num_subgraphs': 1, 'infer_time_core_ms': 255.939395, 'infer_time_subgraph_ms': 255.925773, 'ddr_transfer_mb': 0.0, 'perfsim_time_ms': 0.0, 'perfsim_ddr_transfer_mb': 0.0, 'perfsim_gmacs': 0.0}




#### Package artifacts

Package the artifacts into a .tar.gz file, keeping only the necessary files for inference.

In [12]:
out_dir = f'{work_dir}_package'
tools.package_artifacts(settings, work_dir, out_dir)

packaging artifacts to /tmp/tmp101src05/modelartifacts/32bits_package please wait...
SUCCESS:20220329-161042: finished packaging - /tmp/tmp101src05/modelartifacts/32bits/cl-mnv2_tflitert_imagenet1k_tf1-models_mobilenet_v2_1.0_224_tflite


#### Download

Download the packaged .tar.gz artifact

TODO: add a download link here, that the user can click to download the packaged artifact .tar.gz file.

In [13]:
print(f'download the atricats files from the folder: {out_dir}')
print(os.listdir(out_dir))

download the atricats files from the folder: /tmp/tmp101src05/modelartifacts/32bits_package
['artifacts.yaml', 'artifacts.list', 'extract.sh', 'cl-mnv2_tflitert_imagenet1k_tf1-models_mobilenet_v2_1.0_224_tflite.tar.gz']


#### Cleanup

Remove the temporary folders that we created

In [14]:
# cleanup
modelartifacts_tempdir.cleanup()